In [181]:
import pandas as pd
import psycopg2 as pg2
import snowflake.connector as sf
import os
import credentials
from snowflake.connector.pandas_tools import write_pandas


### Read the datasets

In [82]:
# get the drivers dataset
drivers = pd.read_csv("datasets/cleaned_drivers.csv")

In [83]:
drivers.columns = map(lambda x: str(x).upper(), drivers.columns)
drivers.head()

,DRIVERID,DRIVER_CODE,DRIVER_FORENAME,DRIVER_SURNAME,DRIVER_DOB,DRIVER_NATIONALITY,DRIVER_HOME
0,1,HAM,Lewis,Hamilton,1985-01-07,British,UK
1,2,HEI,Nick,Heidfeld,1977-05-10,German,Germany
2,3,ROS,Nico,Rosberg,1985-06-27,German,Germany
3,4,ALO,Fernando,Alonso,1981-07-29,Spanish,Spain
4,5,KOV,Heikki,Kovalainen,1981-10-19,Finnish,Finland


In [84]:
drivers.columns

Index(['DRIVERID', 'DRIVER_CODE', 'DRIVER_FORENAME', 'DRIVER_SURNAME',
       'DRIVER_DOB', 'DRIVER_NATIONALITY', 'DRIVER_HOME'],
      dtype='object')

In [85]:
constructors= pd.read_csv("datasets/cleaned_constructors.csv")

In [86]:
constructors.columns = map(lambda x: str(x).upper(), constructors.columns)
constructors.head()

,CONSTRUCTORID,CONSTRUCTOR_NAME,CONSTRUCTOR_NATIONALITY,CONSTRUCTOR_HOME
0,1,McLaren,British,UK
1,2,BMW Sauber,German,Germany
2,3,Williams,British,UK
3,4,Renault,French,France
4,5,Toro Rosso,Italian,Italy


In [87]:
circuits = pd.read_csv("datasets/cleaned_circuits.csv")

In [88]:
circuits.columns = map(lambda x: str(x).upper(), circuits.columns)
circuits.head()

,CIRCUITID,CIRCUIT_NAME,CIRCUIT_LOCATION,CIRCUIT_COUNTRY,CIRCUIT_NATIONALITY
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,Australian
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,Malaysian
2,3,Bahrain International Circuit,Sakhir,Bahrain,Bahraini
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,Spanish
4,5,Istanbul Park,Istanbul,Turkey,Turkish


In [89]:
races = pd.read_csv("datasets/cleaned_races.csv")

In [90]:
races.columns = map(lambda x: str(x).upper(), races.columns)
races.head()

,RACEID,YEAR,ROUND,CIRCUITID
0,1,2009,1,1
1,2,2009,2,2
2,3,2009,3,17
3,4,2009,4,3
4,5,2009,5,4


In [91]:
constructor_results = pd.read_csv("datasets/cleaned_constructor_results.csv")


In [92]:
constructor_results.columns = map(lambda x: str(x).upper(), constructor_results.columns)
constructor_results.head()

,CONSTRUCTORRESULTSID,RACEID,CONSTRUCTORID,POINTS
0,1,18,1,14.0
1,2,18,2,8.0
2,3,18,3,9.0
3,4,18,4,5.0
4,5,18,5,2.0


In [139]:
constructor_results = constructor_results[['RACEID','CONSTRUCTORID','POINTS']]


In [140]:
constructor_results.head()

,RACEID,CONSTRUCTORID,POINTS
0,18,1,14.0
1,18,2,8.0
2,18,3,9.0
3,18,4,5.0
4,18,5,2.0


In [93]:
constructor_standings = pd.read_csv("datasets/cleaned_constructor_standings.csv")

In [94]:
constructor_standings.columns = map(lambda x: str(x).upper(), constructor_standings.columns)
constructor_standings.head()

,CONSTRUCTORSTANDINGSID,RACEID,CONSTRUCTORID,POINTS,POSITION,WINS
0,1,18,1,14.0,1,1
1,2,18,2,8.0,3,0
2,3,18,3,9.0,2,0
3,4,18,4,5.0,4,0
4,5,18,5,2.0,5,0


In [95]:
driver_standings = pd.read_csv("datasets/cleaned_driver_standings.csv")

In [96]:
driver_standings.columns = map(lambda x: str(x).upper(), driver_standings.columns)
driver_standings.head()

,DRIVERSTANDINGSID,RACEID,DRIVERID,POINTS,POSITION,WINS
0,1,18,1,10.0,1,1
1,2,18,2,8.0,2,0
2,3,18,3,6.0,3,0
3,4,18,4,5.0,4,0
4,5,18,5,4.0,5,0


In [145]:
driver_standings= driver_standings.rename(columns={'POINTS':'DRIVER_TOTAL_POINTS','POSITION':'DRIVER_POSITION','WINS':'DRIVER_SEASON_WINS'})
driver_standings.head()

,DRIVERSTANDINGSID,RACEID,DRIVERID,DRIVER_TOTAL_POINTS,DRIVER_POSITION,DRIVER_SEASON_WINS
0,1,18,1,10.0,1,1
1,2,18,2,8.0,2,0
2,3,18,3,6.0,3,0
3,4,18,4,5.0,4,0
4,5,18,5,4.0,5,0


In [97]:
results = pd.read_csv("datasets/cleaned_results.csv")

In [149]:
results.columns = map(lambda x: str(x).upper(), results.columns)
results.head()


,RESULTID,RACEID,DRIVERID,CONSTRUCTORID,GRID_POSITION,POSITION,POINTS,LAPS,FASTESTLAP,FASTESTLAPRANK,FASTESTLAPTIME,FASTESTLAPSPEED,STATUSID
0,1,18,1,1,1,1,10.0,58,39,2,0 days 00:01:27.452000,218.300,1
1,2,18,2,2,5,2,8.0,58,41,3,0 days 00:01:27.739000,217.586,1
2,3,18,3,3,7,3,6.0,58,41,5,0 days 00:01:28.090000,216.719,1
3,4,18,4,4,11,4,5.0,58,58,7,0 days 00:01:28.603000,215.464,1
4,5,18,5,1,3,5,4.0,58,43,1,0 days 00:01:27.418000,218.385,1


In [151]:
results= results.rename(columns={'GRID_POSITION':'RESULT_GRID_POSITION','POSITION':'RESULT_POSITION','POINTS':'RESULT_POINTS','LAPS':'NB_LAPS','FASTESTLAP':'RESULT_FASTESTLAP','FASTESTLAPRANK':'RESULT_FASTESTLAPRANK', 'FASTESTLAPTIME':'RESULT_FASTESTLAPTIME', 'FASTESTLAPSPEED':'RESULT_FASTESTLAPSPEED'})
results.head()


,RESULTID,RACEID,DRIVERID,CONSTRUCTORID,RESULT_GRID_POSITION,RESULT_POSITION,RESULT_POINTS,NB_LAPS,RESULT_FASTESTLAP,RESULT_FASTESTLAPRANK,RESULT_FASTESTLAPTIME,RESULT_FASTESTLAPSPEED,STATUSID
0,1,18,1,1,1,1,10.0,58,39,2,0 days 00:01:27.452000,218.300,1
1,2,18,2,2,5,2,8.0,58,41,3,0 days 00:01:27.739000,217.586,1
2,3,18,3,3,7,3,6.0,58,41,5,0 days 00:01:28.090000,216.719,1
3,4,18,4,4,11,4,5.0,58,58,7,0 days 00:01:28.603000,215.464,1
4,5,18,5,1,3,5,4.0,58,43,1,0 days 00:01:27.418000,218.385,1


In [99]:
status = pd.read_csv("datasets/cleaned_status.csv")

In [100]:
status.columns = map(lambda x: str(x).upper(), status.columns)
status.head()

,STATUSID,STATUS
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [101]:
sprint_results = pd.read_csv("datasets/cleaned_sprint_results.csv")

In [102]:
sprint_results.columns = map(lambda x: str(x).upper(), sprint_results.columns)
sprint_results.head()

,RESULTID,RACEID,DRIVERID,CONSTRUCTORID,GRID_POSITION,POSITION,POINTS,LAPS,FASTESTLAP,FASTESTLAPTIME,STATUSID,FASTESTLAPRANK
0,1,1061,830,9,2,1,3,17,14,0 days 00:01:30.013000,1,3
1,2,1061,1,131,1,2,2,17,17,0 days 00:01:29.937000,1,1
2,3,1061,822,131,3,3,1,17,17,0 days 00:01:29.958000,1,2
3,4,1061,844,6,4,4,0,17,16,0 days 00:01:30.163000,1,4
4,5,1061,846,1,6,5,0,17,16,0 days 00:01:30.566000,1,5


In [147]:
sprint_results= sprint_results.rename(columns={'GRID_POSITION':'SPRINT_GRID_POSITION','POSITION':'SPRINT_POSITION','POINTS': 'SPRINT_POINTS','LAPS':'SPRINT_LAPS','FASTESTLAP':'SPRINT_FASTESTLAP','FASTESTLAPTIME':'SPRINT_FASTESTLAPTIME','FASTESTLAPRANK':'SPRINT_FASTESTLAPRANK'})
sprint_results.head()


,RESULTID,RACEID,DRIVERID,CONSTRUCTORID,SPRINT_GRID_POSITION,SPRINT_POSITION,SPRINT_POINTS,SPRINT_LAPS,SPRINT_FASTESTLAP,SPRINT_FASTESTLAPTIME,STATUSID,SPRINT_FASTESTLAPRANK
0,1,1061,830,9,2,1,3,17,14,0 days 00:01:30.013000,1,3
1,2,1061,1,131,1,2,2,17,17,0 days 00:01:29.937000,1,1
2,3,1061,822,131,3,3,1,17,17,0 days 00:01:29.958000,1,2
3,4,1061,844,6,4,4,0,17,16,0 days 00:01:30.163000,1,4
4,5,1061,846,1,6,5,0,17,16,0 days 00:01:30.566000,1,5


In [103]:
lap_times = pd.read_csv("datasets/cleaned_lap_times.csv")

In [104]:
lap_times.columns = map(lambda x: str(x).upper(), lap_times.columns)
lap_times.head()

,RACEID,DRIVERID,LAP,POSITION,LAP_TIME
0,841,20,1,1,0 days 00:01:38.109000
1,841,20,2,1,0 days 00:01:33.006000
2,841,20,3,1,0 days 00:01:32.713000
3,841,20,4,1,0 days 00:01:32.803000
4,841,20,5,1,0 days 00:01:32.342000


In [153]:
lap_times= lap_times.rename(columns={'POSITION':'LAP_POSITION'})
lap_times.head()

,RACEID,DRIVERID,LAP,LAP_POSITION,LAP_TIME
0,841,20,1,1,0 days 00:01:38.109000
1,841,20,2,1,0 days 00:01:33.006000
2,841,20,3,1,0 days 00:01:32.713000
3,841,20,4,1,0 days 00:01:32.803000
4,841,20,5,1,0 days 00:01:32.342000


In [105]:
pit_stops = pd.read_csv("datasets/cleaned_pit_stops.csv")

In [106]:
pit_stops.columns = map(lambda x: str(x).upper(), pit_stops.columns)
pit_stops.head()

,RACEID,DRIVERID,STOP,LAP,MILLISECONDS
0,841,153,1,1,26898
1,841,30,1,1,25021
2,841,17,1,11,23426
3,841,4,1,12,23251
4,841,13,1,13,23842


In [154]:
pit_stops= pit_stops.rename(columns={'STOP':'NB_STOP','LAP':'STOPS_LAP'})
pit_stops.head()

,RACEID,DRIVERID,NB_STOP,STOPS_LAP,MILLISECONDS
0,841,153,1,1,26898
1,841,30,1,1,25021
2,841,17,1,11,23426
3,841,4,1,12,23251
4,841,13,1,13,23842


In [107]:
qualifying = pd.read_csv("datasets/cleaned_qualifying.csv")

In [108]:
qualifying.columns = map(lambda x: str(x).upper(), qualifying.columns)
qualifying.head()

,QUALIFYID,RACEID,DRIVERID,CONSTRUCTORID,NUMBER,POSITION,Q1,Q2,Q3
0,1,18,1,1,22,1,0 days 00:01:26.572000,0 days 00:01:25.187000,0 days 00:01:26.714000
1,2,18,9,2,4,2,0 days 00:01:26.103000,0 days 00:01:25.315000,0 days 00:01:26.869000
2,3,18,5,1,23,3,0 days 00:01:25.664000,0 days 00:01:25.452000,0 days 00:01:27.079000
3,4,18,13,6,2,4,0 days 00:01:25.994000,0 days 00:01:25.691000,0 days 00:01:27.178000
4,5,18,2,2,3,5,0 days 00:01:25.960000,0 days 00:01:25.518000,0 days 00:01:27.236000


In [155]:
qualifying= qualifying.rename(columns={'NUMBER':'QUALIFYING_NUMBER','POSITION':'QUALIFYING_POSITION'})
qualifying.head()

,QUALIFYID,RACEID,DRIVERID,CONSTRUCTORID,QUALIFYING_NUMBER,QUALIFYING_POSITION,Q1,Q2,Q3
0,1,18,1,1,22,1,0 days 00:01:26.572000,0 days 00:01:25.187000,0 days 00:01:26.714000
1,2,18,9,2,4,2,0 days 00:01:26.103000,0 days 00:01:25.315000,0 days 00:01:26.869000
2,3,18,5,1,23,3,0 days 00:01:25.664000,0 days 00:01:25.452000,0 days 00:01:27.079000
3,4,18,13,6,2,4,0 days 00:01:25.994000,0 days 00:01:25.691000,0 days 00:01:27.178000
4,5,18,2,2,3,5,0 days 00:01:25.960000,0 days 00:01:25.518000,0 days 00:01:27.236000


### create a connection with snowflake

In [182]:
user=credentials.user
password=credentials.password
account=credentials.account
database='F1DB'
warehouse='COMPUTE_WH'
schema='PUBLIC'
role='ACCOUNTADMIN'

In [110]:
conn = sf.connect(user = user,
           password = password,
           account = account,
    )

In [111]:
def run_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()

In [112]:
sql = f'use warehouse {warehouse}'
run_query(conn, sql)

In [113]:
try:
    warehouse_sql = 'use warehouse {}'.format(warehouse)
    run_query(conn, warehouse_sql)
    
    try:
        sql = 'alter warehouse {} resume'.format(warehouse)
        run_query(conn, sql)
    except:
        pass
    
    sql = 'use database {}'.format(database)
    
    #sql = 'create database {}'.format(database)
    run_query(conn, sql)
    
    sql = 'use role {}'.format(role)
    run_query(conn, sql)
    
    sql = f'use schema {schema}'
    run_query(conn, sql)
    

except Exception as e:
    print(e)

### Create databases

In [160]:
#sql = 'create  table {}'.format(tablename)
sql = (""" create table if not exists Drivers(
driverId int PRIMARY KEY,
driver_code varchar,
driver_forename varchar, 
driver_surname varchar,
driver_dob varchar, 
driver_nationality varchar, 
driver_home varchar

) """)

run_query(conn, sql)

In [161]:
sql = (""" create table if not exists Constructor(
constructorId int PRIMARY KEY,
constructor_name varchar,
constructor_nationality varchar,
constructor_home varchar

) """)
run_query(conn, sql)

In [162]:
sql = (""" create table if not exists Circuit(
circuitId int PRIMARY KEY,
circuit_name varchar,
circuit_location varchar,
circuit_country varchar,
circuit_nationality varchar

) """)
run_query(conn, sql)

In [163]:
sql = (""" create table if not exists Races(
raceId int PRIMARY KEY,
year int,
round int,
circuitId int,
FOREIGN KEY (circuitId) REFERENCES Circuit(circuitId)

) """)
run_query(conn, sql)

In [164]:
sql = (""" create table if not exists ConstructorResults(
raceId int,
constructorId int,
points int,
PRIMARY KEY (raceId,constructorId), 
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (constructorId) REFERENCES Constructor(constructorId)

) """)
run_query(conn, sql)

In [165]:
sql = (""" create table if not exists ConstructorStandings(
constructorStandingsId int PRIMARY KEY,
raceId int,
constructorId int,
points int,
position int,
wins int,
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (constructorId) REFERENCES Constructor(constructorId)

) """)
run_query(conn, sql)

In [166]:
sql = (""" create table if not exists DriverStandings(
driverStandingsId int PRIMARY KEY,
raceId int ,
driverId int,
driver_total_points int,
driver_position int,
driver_season_wins int,
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId)
) """)
run_query(conn, sql)

In [167]:
sql = (""" create table if not exists Status(
statusId int PRIMARY KEY,
status varchar
) """)
run_query(conn, sql)

In [174]:
sql = (""" create table if not exists Results(
resultId int PRIMARY KEY,
raceId int,
driverId int,
constructorId int,
result_grid_position int,
result_position int,
result_points int,
nb_laps int,
result_fastestLap int,
result_fastestLapRank int,
result_fastestLapTime varchar,
result_fastestLapSpeed float,
statusId int,
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId),
FOREIGN KEY (constructorId) REFERENCES Constructor(constructorId),
FOREIGN KEY (statusId) REFERENCES Status(statusId)

) """)
run_query(conn, sql)

In [169]:
sql = (""" create table if not exists SprintResults(
resultId int,
raceId int,
driverId int,
constructorId int,
sprint_grid_position int,
sprint_position int,
sprint_points int,
sprint_laps int,
sprint_fastestLap int,
sprint_fastestLapTime varchar,
statusId int,
sprint_fastestLapRank int,
PRIMARY KEY (resultId, raceId, driverId),
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId),
FOREIGN KEY (resultId) REFERENCES Results(resultId),
FOREIGN KEY (statusId) REFERENCES Status(statusId)


) """)
run_query(conn, sql)

In [170]:
sql = (""" create table if not exists Qualifying(
qualifyId int PRIMARY KEY,
raceId int,
driverId int,
constructorId int,
qualifying_number int,
qualifying_position int,
q1 varchar,
q2 varchar,
q3 varchar,
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId),
FOREIGN KEY (constructorId) REFERENCES Constructor(constructorId)

) """)
run_query(conn, sql)

In [171]:
sql = (""" create table if not exists PitStops(
raceId int,
driverId int,
nb_stop int,
stops_lap int,
milliseconds int,
PRIMARY KEY (raceId, driverId, nb_stop),
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId)

) """)
run_query(conn, sql)

In [172]:
sql = (""" create table if not exists LapTimes(
raceId int,
driverId int,
lap int,
lap_position int,
lap_time varchar,

PRIMARY KEY (raceId, driverId, lap),
FOREIGN KEY (raceId) REFERENCES Races(raceId),
FOREIGN KEY (driverId) REFERENCES Drivers(driverId)

) """)
run_query(conn, sql)

### Insert into the tables

In [175]:
write_pandas(conn, drivers, table_name='DRIVERS')
write_pandas(conn, constructors, table_name='CONSTRUCTOR')
write_pandas(conn, circuits, table_name='CIRCUIT')
write_pandas(conn, races, table_name='RACES')
write_pandas(conn, constructor_results, table_name='CONSTRUCTORRESULTS')
write_pandas(conn, constructor_standings, table_name='CONSTRUCTORSTANDINGS')
write_pandas(conn, driver_standings, table_name='DRIVERSTANDINGS')
write_pandas(conn, results, table_name='RESULTS')
write_pandas(conn, status, table_name='STATUS')
write_pandas(conn, sprint_results, table_name='SPRINTRESULTS')
write_pandas(conn, lap_times, table_name='LAPTIMES')
write_pandas(conn, pit_stops, table_name='PITSTOPS')
write_pandas(conn, qualifying, table_name='QUALIFYING')

(True,
 1,
 26080,
 [('ezibfwsngz/file0.txt',
   'LOADED',
   26080,
   26080,
   1,
   0,
   None,
   None,
   None,
   None)])